In [ ]:
from pathlib import Path

import geopandas
import pandas
import snkit

from pyproj import Geod

In [ ]:
# set up variables for incoming and processed data paths
project_path = Path().resolve().parent # assume we're running from the project scripts directory, so get the parent
incoming_data_path = project_path / "incoming_data"
processed_data_path = project_path / "processed_data"

In [65]:
# list layers in roads OSM data
geopandas.list_layers(incoming_data_path / "starter-data-kit/data/COD/openstreetmap/openstreetmap_roads-tertiary__COD.gpkg")

,name,geometry_type
0,points,Point
1,lines,LineString
2,multilinestrings,MultiLineString
3,multipolygons,MultiPolygon
4,other_relations,GeometryCollection


In [68]:
# read in the road lines
roads_df = geopandas.read_file(incoming_data_path / "starter-data-kit/data/COD/openstreetmap/openstreetmap_roads-tertiary__COD.gpkg", layer="lines")

In [69]:
# look at set of surface classes in the data
roads_df.surface.unique()

array(['asphalt', None, 'paved', 'sand', 'unpaved', 'dirt', 'compacted',
       'paving_stones', 'ground', 'clay', 'grass', 'gravel', 'JOG',
       'concrete', 'wood', 'earth', 'wood:lanes', 'metal',
       'concrete:plates'], dtype=object)

In [70]:
# simplify surface classes into paved/unpaved
classify_surface = {
    'asphalt': 'paved',
    None: None,
    'paved': 'paved',
    'sand': 'unpaved',
    'unpaved': 'unpaved',
    'dirt': 'unpaved',
    'compacted': 'unpaved',
    'paving_stones': 'paved',
    'ground': 'unpaved',
    'clay': 'unpaved',
    'grass': 'unpaved',
    'gravel': 'unpaved',
    'JOG': 'paved',
    'concrete': 'paved',
    'wood': 'unpaved',
    'earth': 'unpaved',
    'wood:lanes': 'unpaved',
    'metal': 'paved',
    'concrete:plates': 'paved',
}
roads_df['paved'] = roads_df.surface.map(classify_surface)

In [71]:
# look at distribution of paved/unpaved in the data
roads_df[~roads_df.paved.isna()][["highway","paved","osm_id"]].groupby(["highway","paved"]).count()

osm_id
highway        paved          
primary        paved       200
               unpaved    1503
primary_link   paved        11
               unpaved      11
secondary      paved       215
               unpaved    3090
secondary_link paved         4
               unpaved      12
tertiary       paved       357
               unpaved    5985
tertiary_link  paved         2
               unpaved      15
trunk          paved       884
               unpaved    1169
trunk_link     paved        14

In [72]:
# estimate paved for trunk and primary, unpaved otherwise
def estimate_paved(highway):
    if highway in {"primary", "primary_link", "trunk", "trunk_link"}:
        return "paved"
    return "unpaved"

roads_df.loc[roads_df.surface.isna(), "paved"] = roads_df.loc[roads_df.surface.isna(), "highway"].apply(estimate_paved)

In [73]:
# blanket assumption of two lanes
roads_df.lanes = roads_df.lanes.fillna(2)

## Set up road network

In [74]:
road_edges = roads_df[["osm_id","name","ref","highway","lanes","paved","geometry"]].copy()
road_edges

,osm_id,name,ref,highway,lanes,paved,geometry
0,4392435,Boulevard Sendwe,None,trunk,3,paved,"LINESTRING (15.32056 -4.33438, 15.32082 -4.334..."
1,4394364,Rond-Point Ngaba,RN1,trunk,2,paved,"LINESTRING (15.31501 -4.38958, 15.31496 -4.389..."
2,4400935,Avenue By Pass,RN1,trunk,2,paved,"LINESTRING (15.32487 -4.39264, 15.32503 -4.392..."
3,4400953,Avenue Colonel Mondjiba,None,primary,3,paved,"LINESTRING (15.27436 -4.32077, 15.27434 -4.320..."
4,4400958,Avenue de la Libération,None,primary,3,paved,"LINESTRING (15.2923 -4.31672, 15.29226 -4.3166..."
...,...,...,...,...,...,...,...
23463,1375064349,None,None,tertiary,2,unpaved,"LINESTRING (19.90965 -4.91978, 19.90944 -4.919..."
23464,1375064353,None,None,tertiary,2,unpaved,"LINESTRING (19.91791 -4.92505, 19.9179 -4.9242..."
23465,1375432402,None,None,tertiary,2,unpaved,"LINESTRING (16.20629 -3.36985, 16.20627 -3.370..."
23466,1375438291,None,None,tertiary,2,unpaved,"LINESTRING (16.2451 -3.53315, 16.24526 -3.5331..."


In [ ]:
network = snkit.Network(edges=road_edges)
split = snkit.network.split_edges_at_intersections(network)
with_endpoints = snkit.network.add_endpoints(split)
with_topology = snkit.network.add_topology(snkit.network.add_ids(with_endpoints))

In [ ]:
geod = Geod(ellps="WGS84")
with_topology.edges.length_m = with_topology.edges.geometry.apply(geod.geometry_length)

In [ ]:
road_network_path = processed_data_path / "networks" / "transport" / "road.2025-04-30.gpkg"
road_network_path.parent.mkdir(exist_ok=True, parents=True)
with_topology.to_file(road_network_path, nodes_layer="road_nodes", edges_layer="road_edges")

In [ ]:
road_graph = snkit.network.to_networkx(with_topology, weight_col="length_m")

In [ ]:
f_vuln_depth = pandas.read_excel("Table_D2_Hazard_Fragility_and_Vulnerability_Curves_V1.1.0.xlsx", sheet_name="F_Vuln_Depth")
f_vuln_depth